In [9]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import glob
import matplotlib.pyplot as plt


import os
import seaborn as sns

# from ripple_heterogeneity.utils import loading,functions
from neuro_py.io import loading
from neuro_py.process.intervals import in_intervals, truncate_epoch
import nelpy as nel  # main nelpy imports




In [1]:
def process_binary(basepath,ext = "dat"):
    nChannels, fs, fs_dat, _ = loading.loadXML(basepath)
        
    if ext == 'dat':
        fs = fs_dat
        # load dat file
        signal, ts = loading.loadLFP(basepath, n_channels=nChannels,
                        channel=None,
                        frequency=fs,
                        ext = ext)
    else: 
        # load lfp
        signal, ts = loading.loadLFP(basepath, n_channels=nChannels,
                        channel=None,
                        frequency=fs,
                        ext = ext)
    
    return signal, ts, fs

In [16]:
# set basepath 
basepath = os.path.normpath(r'Y:\laura_berkowitz\app_ps1_ephys\data\hpc10\hpc10_day35_220607_120154')

# load the dat file
dat, ts, fs = process_binary(basepath)

# load sleep states 
sleep_ep = loading.load_SleepState_states(basepath)
rem_epochs = nel.EpochArray(sleep_ep["REMstate"])
theta_epochs = nel.EpochArray(sleep_ep["THETA"])

# truncate to 2 minute 
rem_epochs = truncate_epoch(rem_epochs, time=120)
theta_epochs = truncate_epoch(theta_epochs, time=120)


rem = in_intervals(ts, rem_epochs.data)
theta = in_intervals(ts, theta_epochs.data)

In [11]:
rem_epochs

<EpochArray at 0x225cda02850: 2 epochs> of length 2:00 minutes

In [17]:
# dat is loaded as memmap, so we can index as if it were a numpy array
rip_channel = dat[rem,4]
sw_channel = dat[rem,55]
noise_channel = dat[rem,47]

In [20]:
# dat is loaded as memmap, so we can index as if it were a numpy array
rip_channel = dat[rem,63]
sw_channel = dat[rem,2]
noise_channel = dat[rem,13]

In [21]:
# create a dataframe
df = pd.DataFrame(columns=['rip','sw','noise'])
df['rip'] = rip_channel
df['sw'] = sw_channel
df['noise'] = noise_channel

# save the dataframe
df.to_csv(os.path.join("Y:/laura_berkowitz/app_ps1_ephys/analysis","hpc10_day35_rem_dat.csv"))